In [1]:
# SETUP ===================================
import jupyter
import warnings

from src.utils.system import boot, Notify

boot()
warnings.filterwarnings("ignore")



# PACKAGES ================================
import os
import torch
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import gymnasium as gym
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import  RobustScaler

# FRAMEWORK STUFF =========================
from src.defaults import TOP2_STOCK_BY_SECTOR, FEATURE_COLS,EPISODE_LENGTH
from src.data.feature_pipeline import load_base_dataframe
from src.experiments.experiment_tracker import ExperimentTracker
from src.env.base_timeseries_trading_env import BaseSequenceAwareTradingEnv,SequenceAwareAlphaTradingEnv,SequenceAwareBaselineTradingAgent,SequenceAwareCalmarTradingEnv,SequenceAwareCumulativeTradingEnv,SequenceAwareDrawdownTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareHybridTradingEnv,SequenceAwareSharpeTradingEnv,SequenceAwareSortinoTradingEnv

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy



In [2]:

# ========== CONFIG ==========
EXPERIENCE_NAME = "core_sequence_aware_agent_design"
RESULTS_PATH = f"data/experiments/{EXPERIENCE_NAME}_barebones_results.csv"
N_EPISODES = 20
N_SEEDS = 3
N_EVAL_EPISODES = 3
AGENT_TYPES = ['mlp', 'lstm', 'transformer_single', 'transformer_multi']
WINDOW_LENGTH = 10  # or any value you want
TOTAL_TIMESTEPS = EPISODE_LENGTH * 150
N_STEPS = EPISODE_LENGTH * 2

TRANSACTION_COST = 0

CONFIG = {
    "batch_size": EPISODE_LENGTH,
    "n_steps": 128,
    "total_timesteps": TOTAL_TIMESTEPS,   
}


"""
features_extractor_kwargs={
    'window_length': WINDOW_LENGTH,
    'n_features': len(FEATURE_COLS),
    'd_model': 32,
    'nhead': ...,
    'num_layers': ...,
}
"""

# --- Load data ---
ohlcv_df = load_base_dataframe()

# --- Experiment tracker ---
experiment_tracker = ExperimentTracker(EXPERIENCE_NAME)



In [3]:
def make_env(df, ticker, feature_cols, episode_length, window_length):
    df_ticker = df[df['symbol'] == ticker].copy()
    return CumulativeTradingEnv(
        df=df_ticker,
        feature_cols=feature_cols,
        episode_length=episode_length,
        transaction_cost=TRANSACTION_COST,
        window_length=window_length,
    )

In [4]:
class TransformerExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, window_length, n_features, d_model=32, nhead=1, num_layers=1):
        super().__init__(observation_space, features_dim=d_model)
        self.window_length = window_length
        self.n_features = n_features
        self.embedding = nn.Linear(n_features, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, obs):
        # obs: [batch, window_length * n_features]
        batch = obs.shape[0]
        # reshape flat vector to (batch, window_length, n_features)
        x = obs.view(batch, self.window_length, self.n_features)
        x = self.embedding(x)      # (batch, window_length, d_model)
        x = x.permute(1, 0, 2)    # (window_length, batch, d_model)
        x = self.transformer(x)    # (window_length, batch, d_model)
        # Use last token as pooled output
        return x[-1]              # (batch, d_model)

In [5]:
class TransformerPolicy(ActorCriticPolicy):
    def __init__(self, *args, nhead=1, num_layers=1, window_length=WINDOW_LENGTH, n_features=2, **kwargs):
        super().__init__(
            *args,
            features_extractor_class=TransformerExtractor,
            features_extractor_kwargs={
                'window_length': window_length,
                'n_features': n_features,
                'd_model': 32,
                'nhead': nhead,
                'num_layers': num_layers,
            },
            **kwargs
        )

In [6]:
# Test 1: Output Shapes

# Test windowed obs shape (flat vs. 2D)
df = ohlcv_df.copy()
feature_cols = FEATURE_COLS
env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=True
)
obs, _ = env.reset()
print("2D window shape:", obs.shape)  # Expect (5, obs_dim)

env_flat = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=False
)
obs_flat, _ = env_flat.reset()
print("Flat window shape:", obs_flat.shape)  # Expect (5*obs_dim,)


2D window shape: (10, 25)
Flat window shape: (250,)


In [7]:
done = False
i = 0
while not done:
    obs,reward,done,_,info = env.step(1)
    i+=1
i

100

In [9]:
len(info["returns"]),env.episode_length

(100, 100)

# Unit tests:
1. Output Shapes
2. Window Consistency (Padding at Episode Start)
3. Step Through Environment
4. SB3 Policy Compatibility
5. Transformer Policy Compatibility
6. Action Space and Reward Consistency
7. Episode Generator
8. Is able to learn

In [10]:
# Test 1: Output Shapes

# Test windowed obs shape (flat vs. 2D)
df = ohlcv_df.copy()
feature_cols = FEATURE_COLS
env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=True
)
obs, _ = env.reset()
print("2D window shape:", obs.shape)  # Expect (5, obs_dim)

env_flat = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=False
)
obs_flat, _ = env_flat.reset()
print("Flat window shape:", obs_flat.shape)  # Expect (5*obs_dim,)


2D window shape: (10, 25)
Flat window shape: (250,)


In [11]:
# Test 2: Window consistency
env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=True
)
obs, _ = env.reset()
assert np.allclose(obs[0], obs[1]), "Padding at start should repeat first row"
assert obs.shape == (WINDOW_LENGTH, len(feature_cols) + len(env.internal_features))
print("Padding and shape OK")


Padding and shape OK


In [12]:
# Test 3: Step Through Environment

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=True
)
obs, _ = env.reset()
for i in range(8):
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    print(f"Step {i} | Obs shape: {obs.shape} | Reward: {reward:.5f}")
    if done:
        print("Episode done:", info)
        break

Step 0 | Obs shape: (10, 25) | Reward: -0.00000
Step 1 | Obs shape: (10, 25) | Reward: -0.00168
Step 2 | Obs shape: (10, 25) | Reward: -0.00290
Step 3 | Obs shape: (10, 25) | Reward: -0.00425
Step 4 | Obs shape: (10, 25) | Reward: 0.00380
Step 5 | Obs shape: (10, 25) | Reward: 0.00045
Step 6 | Obs shape: (10, 25) | Reward: -0.00605
Step 7 | Obs shape: (10, 25) | Reward: 0.00463


In [13]:
# SB3 Policy Compatibility
# Train an MLP agent on env with return_sequences=False (flat). 

from stable_baselines3 import PPO

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=False
)
from stable_baselines3.common.vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda: env])

model = PPO("MlpPolicy", vec_env, n_steps=, batch_size=4, verbose=0)
model.learn(total_timesteps=TOTAL_TIMESTEPS)
print("SB3 PPO MLP works!")

KeyboardInterrupt: 

In [ ]:
# Test 5: Transformer Policy Compatibility
# Make sure custom transformer can process the 2D obs by running a forward pass 
# through the extractor to check for shape errors


obs = np.random.randn(2, 5*8).astype(np.float32)  # batch=2, window_length=WINDOW_LENGTH, n_features=8
# Extractor expects (batch, window_length*n_features), will reshape internally.
extractor = TransformerExtractor(
    gym.spaces.Box(-np.inf, np.inf, shape=(5*8,), dtype=np.float32), 5, 8
)
with torch.no_grad():
    torch_out = extractor(torch.from_numpy(obs))
print("Transformer output shape:", torch_out.shape)


In [ ]:
# Test 6: Action Space and Reward Consistency
# mini-episode ti check action output and cumulative reward:

env = BaseSequenceAwareTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, return_sequences=False
)
obs, _ = env.reset()
cumulative = 0
for _ in range(10):
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    cumulative += reward
    if done:
        print("Episode finished | Cumulative reward:", cumulative)
        print("Info dict:", info)
        break

In [ ]:
# Test 7: Episode Generator
# Check that the same seed produces the same episode list across runs.

env = BaseSequenceAwareTradingEnv(df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH)
seq1 = env.generate_episode_sequences(train_steps=TOTAL_TIMESTEPS)
env2 = BaseSequenceAwareTradingEnv(df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH)
seq2 = env2.generate_episode_sequences(train_steps=TOTAL_TIMESTEPS)
assert seq1 == seq2, "Episode sequences should be the same for same seed!"
print("Episode generator determinism OK")

In [ ]:
# Test 8: Learnability
from src.env.realistic_synthetic_environment import realistic_synthetic_market_sample
class RandomAgent:
    def __init__(self, env):
        self.env = env
    def predict(self, obs, *args, **kwargs):
        return self.env.action_space.sample(), {}

class AlwaysLongAgent:
    def __init__(self, env):
        self.env = env
    def predict(self, obs, *args, **kwargs):
        return 1, {}  # Always go long
    
def evaluate_baseline_agent(env, agent, n_episodes=20, episode_sequence=None):
    rewards = []
    if episode_sequence:
        env.set_episode_sequence(episode_sequence)
    for _ in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = agent.predict(obs)
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
    return np.mean(rewards), np.std(rewards)



In [ ]:
from stable_baselines3 import PPO

env = SequenceAwareCumulativeTradingEnv(df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, seed=314)
env.set_episode_sequence(seq)


# Evaluate PPO agent
def evaluate_sb3_agent(env, model, n_episodes=10, episode_sequence=None):
    rewards = []
    if episode_sequence:
        env.set_episode_sequence(episode_sequence)
    for _ in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
    return np.mean(rewards), np.std(rewards)



In [ ]:
from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO

def make_agent(agent_type, env, window_length, feature_cols, **kwargs):
    if agent_type == 'mlp':
        return PPO("MlpPolicy", env, verbose=0, **kwargs)
    elif agent_type == 'lstm':
        return RecurrentPPO("MlpLstmPolicy", env, verbose=0, **kwargs)
        #return PPO("MlpLstmPolicy", env, verbose=0, **kwargs)
    elif agent_type.startswith('transformer'):
        
        n_features = len(feature_cols)
       
        return PPO(
            TransformerPolicy,
            env,
            verbose=0,
            policy_kwargs={
                'window_length': window_length,
                'n_features': env.observation_space.shape[1],
                'nhead': 2,        # set as desired
                'num_layers': 2,   # set as desired
            },
            **kwargs
        )
    else:
        raise ValueError(f"Unknown agent type: {agent_type}")


In [ ]:
def evaluate_agent(env, agent, n_episodes=10, episode_sequence=None, is_sb3=False):
    rewards = []
    if episode_sequence:
        env.set_episode_sequence(episode_sequence)
    for _ in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            if is_sb3:
                action, _ = agent.predict(obs, deterministic=True)
            else:
                action, _ = agent.predict(obs)
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
        rewards.append(total_reward)
    return np.mean(rewards), np.std(rewards)

In [ ]:
# ENVIRONMENT AND SEQUENCES ======================
df = realistic_synthetic_market_sample(n=200)
feature_cols = FEATURE_COLS
env = SequenceAwareCumulativeTradingEnv(
    df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, seed=314)
seq = env.generate_episode_sequences(train_steps=TOTAL_TIMESTEPS)

# Baseline agents =================================
random_agent = RandomAgent(env)
always_long_agent = AlwaysLongAgent(env)
mean_rand, std_rand = evaluate_agent(env, random_agent, n_episodes=20, episode_sequence=seq)
mean_long, std_long = evaluate_agent(env, always_long_agent, n_episodes=20, episode_sequence=seq)
print(f"Random: mean {mean_rand:.4f}, std {std_rand:.4f}")
print(f"Always Long: mean {mean_long:.4f}, std {std_long:.4f}")

# RL agents =======================================
AGENT_TYPES = ['mlp', 'lstm', 'transformer_single', 'transformer_multi']
#AGENT_TYPES = ['transformer_single', 'transformer_multi']
for agent_type in AGENT_TYPES:
    print(f"\nTraining {agent_type} agent...")
    env = SequenceAwareCumulativeTradingEnv(
        df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, seed=314)
    env.set_episode_sequence(seq)
    model = make_agent(agent_type, env, window_length=WINDOW_LENGTH, feature_cols=feature_cols, n_steps=EPISODE_LENGTH, batch_size=4)
    model.learn(total_timesteps=TOTAL_TIMESTEPS)
    mean_rl, std_rl = evaluate_agent(env, model, n_episodes=10, episode_sequence=seq, is_sb3=True)
    print(f"{agent_type} agent: mean {mean_rl:.4f}, std {std_rl:.4f}")


In [ ]:
env.episode_df

In [36]:
# BONUS TEST - Full Episode Learning 
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

class EpisodicPPOTrainer:
    def __init__(self, 
                 env_factory,           # callable, returns fresh env instance
                 policy="MlpPolicy",    # or custom policy class
                 episode_length=100,
                 episodes_per_update=4, # number of full episodes per PPO update
                 total_episodes=1000,
                 verbose=1,
                 agent_kwargs=None):
        self.env_factory = env_factory
        self.episode_length = episode_length
        self.episodes_per_update = episodes_per_update
        self.total_episodes = total_episodes
        self.verbose = verbose
        self.policy = policy
        #self.agent_kwargs = agent_kwargs or {}

        # Build env and agent
        self.env = DummyVecEnv([self.env_factory])
        self.env = DummyVecEnv([self.env_factory])
        steps_per_update = self.episode_length * self.episodes_per_update
        self.agent = PPO(
            policy, 
            self.env,
            n_steps=steps_per_update,
            batch_size=steps_per_update, # so update only at episode boundary
            verbose=verbose
        )

    def train(self, log_callback=None):
        episode_count = 0
        rewards_log = []
        if self.verbose:
            print(f"Training: {self.total_episodes} episodes, updating every {self.episodes_per_update} episodes.")
        while episode_count < self.total_episodes:
            all_obs, all_actions, all_rewards, all_dones, all_values, all_logprobs = [], [], [], [], [], []
            for ep in range(self.episodes_per_update):
                obs = self.env.reset()        # <-- FIX: initialize obs
                done = False

                ep_obs, ep_actions, ep_rewards, ep_dones, ep_values, ep_logprobs = [], [], [], [], [], []
                while not done:
                    # Vectorized env returns obs shape (1, obs_dim)
                    obs_tensor = torch.from_numpy(obs).float()
                    action, _ = self.agent.predict(obs, deterministic=False)
                    action_tensor = torch.from_numpy(action).long()
                    value = self.agent.policy.predict_values(torch.as_tensor(obs)).detach()
                    logprob = self.agent.policy.evaluate_actions(
                        torch.as_tensor(obs), torch.as_tensor(action)
                    )[1].detach()
                    next_obs, reward, done_arr, info = self.env.step(action)
                    done = done_arr[0] if isinstance(done_arr, np.ndarray) else done_arr
                    reward = reward[0] if isinstance(reward, np.ndarray) else reward
                    ep_obs.append(obs)
                    ep_actions.append(action)
                    ep_rewards.append(reward)
                    ep_dones.append(done)
                    ep_values.append(value)
                    ep_logprobs.append(logprob)
                    obs = next_obs
                all_obs.extend(ep_obs)
                all_actions.extend(ep_actions)
                all_rewards.extend(ep_rewards)
                all_dones.extend(ep_dones)
                all_values.extend(ep_values)
                all_logprobs.extend(ep_logprobs)
                rewards_log.append(np.sum(ep_rewards))
                episode_count += 1

                if log_callback is not None:
                    log_callback(episode_count, rewards_log)

            # Fill rollout buffer with these episodes
            self.agent.rollout_buffer.reset()
          
            for i in range(len(all_obs)):
                self.agent.rollout_buffer.add(
                    all_obs[i], all_actions[i], all_rewards[i], all_dones[i], all_values[i], all_logprobs[i]
                )
            # --- Fix logger bug (SB3 expects setup_learn called at least once) ---
            if not hasattr(self.agent, "_logger"):
                self.agent._setup_learn(1)
            self.agent.train()

            if self.verbose:
                print(f"Episode {episode_count}: last episode reward {rewards_log[-1]:.4f}")

        return rewards_log

    def evaluate(self, n_episodes=10):
        rewards = []
        
        for _ in range(n_episodes):
            obs = self.env.reset()
            done = False
            total_reward = 0
            while not done:
                action, _ = self.agent.predict(obs, deterministic=True)
                obs, reward, done,info = self.env.step(action)
                total_reward += reward
            rewards.append(total_reward)
        mean_r = np.mean(rewards)
        std_r = np.std(rewards)
        if self.verbose:
            print(f"Eval: mean reward {mean_r:.4f}, std {std_r:.4f}")
        return mean_r, std_r

# ========== Usage Example ==========

# Example env_factory for your env:
class EnvFactory:
    def __init__(self,
                 env_class,
                 df,
                 feature_cols=None,
                 internal_features=None,
                 episode_length=100,
                 transaction_cost=0.0001,
                 seed=314, 
                 window_length=10,
                 return_sequences=True):
        
        self.df = df.copy()
        self.env_class = env_class
        self.feature_cols = feature_cols
        self.internal_features=internal_features
        self.episode_length=episode_length
        self.transaction_cost=transaction_cost
        self.seed=seed
        self.window_length=window_length
        self.return_sequences=return_sequences
    
    def generate(self):
        return self.env_class(self.df.copy(),
                              feature_cols=self.feature_cols, 
                              internal_features=self.internal_features,
                              episode_length=self.episode_length, 
                              window_length=self.window_length, 
                              transaction_cost=self.transaction_cost,
                              seed=self.seed, 
                 
                              return_sequences=self.return_sequences)

SEED = 314
env_factory = EnvFactory(
    SequenceAwareCumulativeTradingEnv,
    df, 
    feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, seed=SEED,return_sequences=True)

trainer = EpisodicPPOTrainer(
    env_factory=env_factory.generate,
    policy="MlpPolicy",              # Can use custom
    episode_length=100,
    episodes_per_update=4,
    total_episodes=10,
    #agent_kwargs=dict(n_epochs=2, learning_rate=2e-4)  # as you wish
)

trainer.train()
trainer.evaluate()


Using cpu device
Training: 10 episodes, updating every 4 episodes.
Episode 4: last episode reward 0.1170
Episode 8: last episode reward 0.1489
Episode 12: last episode reward 0.2073
Eval: mean reward -0.0627, std 0.0000


(-0.062725686, 7.450581e-09)

In [ ]:
env = SequenceAwareCumulativeTradingEnv(df, feature_cols=feature_cols, episode_length=EPISODE_LENGTH, window_length=WINDOW_LENGTH, seed=314)
env.set_episode_sequence([[df['symbol'].unique()[0],0]])

In [19]:
env.reset()

(array([[ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,
          1.4064915e-01, -6.2961789e-04,  1.7500000e+00, -4.0172415e+00,
          4.1633849e-03,  1.4009227e-02,  1.4009227e-02,  3.1394106e-01,
          3.1839257e-01,  1.8674700e-02,  2.4124569e-01,  1.4954310e-02,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,
          1.4064915e-01, -6.2961789e-04,  1.7500000e+00, -4.0172415e+00,
          4.1633849e-03,  1.4009227e-02,  1.4009227e-02,  3.1394106e-01,
          3.1839257e-01,  1.8674700e-02,  2.4124569e-01,  1.4954310e-02,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,

In [21]:
env.step(1)

(array([[ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,
          1.4064915e-01, -6.2961789e-04,  1.7500000e+00, -4.0172415e+00,
          4.1633849e-03,  1.4009227e-02,  1.4009227e-02,  3.1394106e-01,
          3.1839257e-01,  1.8674700e-02,  2.4124569e-01,  1.4954310e-02,
          1.0000000e+00,  1.0000000e+00, -4.1058646e-03,  0.0000000e+00,
          4.1058646e-03,  1.5286901e-02, -4.1058646e-03,  1.8023000e+02,
          0.0000000e+00],
        [ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,
          1.4064915e-01, -6.2961789e-04,  1.7500000e+00, -4.0172415e+00,
          4.1633849e-03,  1.4009227e-02,  1.4009227e-02,  3.1394106e-01,
          3.1839257e-01,  1.8674700e-02,  2.4124569e-01,  1.4954310e-02,
          1.0000000e+00,  1.0000000e+00, -4.1058646e-03,  0.0000000e+00,
          4.1058646e-03,  1.5286901e-02, -4.1058646e-03,  1.8023000e+02,
          0.0000000e+00],
        [ 5.4095829e-01, -4.6153846e-03,  4.0000000e+00,  1.0000000e+00,